# Autoencoding dataset

In more complex problems, the network architecture has to be deeper, as the relationships might be harder to model. There are some lessons to be learned while working on such examples, so this chapter will be an end-to-end implementation of the autoencoder applied for anomaly detection.

In [3]:
%matplotlib inline

import pandas as pd

transactions_df = pd.read_csv(
    "./dataset/creditcardfraud/creditcard.csv")
transactions_df.head().T

,0,1,2,3,4
Time,0.000000,0.000000,1.000000,1.000000,2.000000
V1,-1.359807,1.191857,-1.358354,-0.966272,-1.158233
V2,-0.072781,0.266151,-1.340163,-0.185226,0.877737
V3,2.536347,0.166480,1.773209,1.792993,1.548718
V4,1.378155,0.448154,0.379780,-0.863291,0.403034
V5,-0.338321,0.060018,-0.503198,-0.010309,-0.407193
V6,0.462388,-0.082361,1.800499,1.247203,0.095921
V7,0.239599,-0.078803,0.791461,0.237609,0.592941
V8,0.098698,0.085102,0.247676,0.377436,-0.270533
V9,0.363787,-0.255425,-1.514654,-1.387024,0.817739


In [40]:
transactions_df.groupby("Class").count()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,284315,284315,284315,284315,284315,284315,284315,284315,284315,284315,...,284315,284315,284315,284315,284315,284315,284315,284315,284315,284315
1,492,492,492,492,492,492,492,492,492,492,...,492,492,492,492,492,492,492,492,492,492


## Scaling the dataset

In [43]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
transactions_scaled_df = pd.DataFrame(min_max_scaler.fit_transform(transactions_df), 
                                      columns=transactions_df.columns)

In [52]:
X_nonfraud = transactions_scaled_df[0 == transactions_scaled_df["Class"]]\
    .drop("Class", axis=1)
X_fraud = transactions_scaled_df[1 == transactions_scaled_df["Class"]]\
    .drop("Class", axis=1)

X_nonfraud.shape, X_fraud.shape

((284315, 30), (492, 30))

## Defining network architecture

In [64]:
from keras.layers import Input, Dense
from keras.models import Model

input_layer = Input(shape=(30, ))
hidden_1 = Dense(15, activation="tanh")(input_layer)
encoded = Dense(10, activation="tanh")(hidden_1)
hidden_2 = Dense(15, activation="tanh")(encoded)
decoded = Dense(30, activation="sigmoid")(hidden_2)
autoencoder = Model(input_layer, decoded)

In [65]:
autoencoder.compile(optimizer="adam", 
                    loss="mean_squared_error")

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(X_nonfraud, test_size=0.25, 
                                   random_state=2019)

In [67]:
autoencoder.fit(X_train, X_train,
                epochs=100,
                batch_size=128,
                shuffle=True,
                validation_data=(X_test, X_test))

Train on 213236 samples, validate on 71079 samples
Epoch 1/100
213236/213236 [==============================] - 2s 9us/step - loss: 0.0035 - val_loss: 0.0015
Epoch 2/100
213236/213236 [==============================] - 2s 9us/step - loss: 0.0011 - val_loss: 8.2554e-04
Epoch 3/100
213236/213236 [==============================] - 2s 9us/step - loss: 7.1851e-04 - val_loss: 6.1745e-04
Epoch 4/100
213236/213236 [==============================] - 2s 9us/step - loss: 5.9110e-04 - val_loss: 5.6129e-04
Epoch 5/100
213236/213236 [==============================] - 2s 9us/step - loss: 4.8390e-04 - val_loss: 3.9797e-04
Epoch 6/100
213236/213236 [==============================] - 2s 10us/step - loss: 3.6912e-04 - val_loss: 3.6014e-04
Epoch 7/100
213236/213236 [==============================] - 2s 11us/step - loss: 3.4364e-04 - val_loss: 3.2660e-04
Epoch 8/100
213236/213236 [==============================] - 2s 9us/step - loss: 3.1192e-04 - val_loss: 3.0083e-04
Epoch 9/100
213236/213236 [============

213236/213236 [==============================] - 2s 10us/step - loss: 2.3669e-04 - val_loss: 2.3533e-04
Epoch 72/100
213236/213236 [==============================] - 2s 11us/step - loss: 2.3647e-04 - val_loss: 2.3527e-04
Epoch 73/100
213236/213236 [==============================] - 3s 13us/step - loss: 2.3636e-04 - val_loss: 2.4011e-04
Epoch 74/100
213236/213236 [==============================] - 2s 9us/step - loss: 2.3626e-04 - val_loss: 2.3463e-04
Epoch 75/100
213236/213236 [==============================] - 2s 9us/step - loss: 2.3596e-04 - val_loss: 2.3423e-04
Epoch 76/100
213236/213236 [==============================] - 2s 9us/step - loss: 2.3590e-04 - val_loss: 2.3739e-04
Epoch 77/100
213236/213236 [==============================] - 3s 12us/step - loss: 2.3587e-04 - val_loss: 2.3480e-04
Epoch 78/100
213236/213236 [==============================] - 2s 10us/step - loss: 2.3547e-04 - val_loss: 2.3383e-04
Epoch 79/100
213236/213236 [==============================] - 2s 11us/step - los

In [58]:
from keras.losses import mean_squared_error
from keras import backend as K

mse = K.eval(mean_squared_error(X_test, 
                                autoencoder.predict(X_test)))
mse_df = pd.DataFrame.from_dict({"mse": mse})
mse_df.describe()

,mse
count,283823.000000
mean,0.001849
std,0.001564
min,0.000178
25%,0.000987
50%,0.001539
75%,0.002359
max,0.268686


In [63]:
mse_fraud = K.eval(mean_squared_error(X_fraud, 
                                      autoencoder.predict(X_fraud)))
mse_fraud_df = pd.DataFrame.from_dict({"mse": mse_fraud})
mse_fraud_df.describe()

,mse
count,492.000000
mean,0.020664
std,0.021598
min,0.000499
25%,0.005255
50%,0.011626
75%,0.029749
max,0.086163
